In [ ]:
!pip install soniox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install pyannote.core
!pip install pyannote.metrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=a397bbfb8e5d7ee6b331539b04ca5c74c991a64905233690f5c37f5b1e772146
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import time
import pandas as pd
import re
from soniox.speech_service import SpeechClient
from soniox.transcribe_file import transcribe_file_async


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/AfriSpeech-Dialog-data/data/diarized_tag_afrispeech_dialog_v1.1_49.csv')

In [ ]:
diarize_ds = dataset[dataset['all_timestamps']==True].reset_index(drop=True)

In [ ]:
#remove the 1, 26, 28 rows with incomplete timestamps
diarize_ds = diarize_ds.drop([1, 26, 28]).reset_index(drop=True)
# print(diarize_ds.iloc[28]['transcript'][200:])

In [ ]:
diarize_ds['path'] = diarize_ds['path'].apply(lambda x: x.replace('data/non-medical', '/content/drive/MyDrive/AfriSpeech-Dialog-data/data/non-medical'))

In [ ]:
import getpass
soniox_api_key = getpass.getpass('Soniox API Key: ')

Soniox API Key: ··········


In [ ]:
def transcribe(audio_path):
    with SpeechClient(api_key=soniox_api_key) as client:
        # print("Uploading file.")
        file_id = transcribe_file_async(
            audio_path,
            client,
            model="en_v2",
            reference_name="test",
            enable_global_speaker_diarization=True
        )
        # print(f"File ID: {file_id}")

        while True:
            # print("Calling GetTranscribeAsyncStatus.")
            status = client.GetTranscribeAsyncStatus(file_id)
            # print(f"Status: {status.status}")
            if status.status in ("COMPLETED", "FAILED"):
                break
            time.sleep(2.0)

        if status.status == "COMPLETED":
            # print("Calling GetTranscribeAsyncResult")
            result = client.GetTranscribeAsyncResult(file_id)
            # print(f"Text: " + "".join(word.text for word in result.words))
        else:
            print(f"Transcription failed with error: {status.error_message}")

        # print("Calling DeleteTranscribeAsyncFile.")
        client.DeleteTranscribeAsyncFile(file_id)
        return result



In [ ]:
test_res = transcribe("/content/drive/MyDrive/AfriSpeech-Dialog-data/data/non-medical/98cc8263-54e8-4101-8942-7ae18f1e082b_7fb3ed3c8a1328bd2d7ea25dbc5a9fdd_j7rw2vU3.wav")

Uploading file.
File ID: 81db67b5-ba88-469c-857e-5f41d683bed7
Calling GetTranscribeAsyncStatus.
Status: QUEUED
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling GetTranscribeAsyncStatus.
Status: TRANSCRIBING
Calling G

In [ ]:
type(test_res)

soniox.speech_service_pb2.Result

In [ ]:
def get_utterances_from_words(word_objects):
    utterances = []
    current_speaker = None
    start_time = None
    end_time = None

    for word in word_objects:
        word_start = word.start_ms / 1000  # Convert ms to seconds
        word_end = word_start + word.duration_ms / 1000  # Calculate the end time

        if current_speaker is None:  # First word case
            current_speaker = word.speaker
            start_time = word_start
            end_time = word_end
        elif word.speaker == current_speaker:
            # Extend the current utterance end time
            end_time = word_end
        else:
            # Append the current utterance and start a new one
            utterances.append((start_time, end_time, f"Speaker {current_speaker}"))
            current_speaker = word.speaker
            start_time = word_start
            end_time = word_end

    # Append the last utterance
    if current_speaker is not None:
        utterances.append((start_time, end_time, f"Speaker {current_speaker}"))

    return utterances



In [ ]:
from tqdm import tqdm

In [ ]:
diarize_ds.head()

In [ ]:
diarize_ds['pred_segments'] = ''
for i, path_ in tqdm(enumerate(diarize_ds['path']), total=len(diarize_ds['path']), desc="Processing"):
    result = transcribe(path_)
    pred_segments = get_utterances_from_words(result.words)

    diarize_ds.at[i, 'pred_segments'] = pred_segments

Processing: 100%|██████████| 30/30 [57:44<00:00, 115.50s/it]


In [ ]:
diarize_ds.head()

,audio_id,path,transcript,age_group,gender,accent,domain,country,audio_duration,doctor_summary,all_timestamps,pred_segments
0,78fb7265-0373-49d5-b933-c28c95cf3eea,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:01:06\r\n[Speaker 1]: Are you there?\r\n00:...,26-40,Male,Yoruba,OSCE-Doctor-Patient,NG,906.433991,"Mr. Ade, a 35 year old unmarried Yoruba male t...",True,"[(0.84, 3.96, Speaker 1), (3.96, 9.12, Speaker..."
1,6d5f1e84-dbc5-42d3-898e-bfe9c2cfe166,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,"00:00:06\r\n[Speaker 1]: Good evening, I'm Deb...",26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,204.622993,Johnny Dobra is a 35-year-old male who present...,True,"[(0.06, 2.76, Speaker 1), (2.76, 6.42, Speaker..."
2,ab8fd39d-52bc-48fa-bae9-ff636627f1e5,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:00:08\r\n[Speaker 1]: My name is Nwachukwu ...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,343.260000,"The patient, a 45 year old female accontant, p...",True,"[(2.88, 10.08, Speaker 1), (10.08, 11.94, Spea..."
3,5129fd8c-7b8c-4d05-a03a-196bcae4deff,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:00:50\r\n[Speaker 1]: My name is Chiamaka D...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,337.587982,"John Peter, a 45-year-old construction worker,...",True,"[(0.66, 4.14, Speaker 1), (4.14, 7.44, Speaker..."
4,58efe9ef-6c76-4d17-b1b2-f397552a0c0e,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:01:13\r\n[Speaker 1]: Good day. I am Doctor...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,213.635986,"Peter, a 45-year-old male, presented to the ho...",True,"[(1.14, 5.58, Speaker 1), (5.58, 7.08, Speaker..."


In [ ]:
def convert_time_to_seconds(timestamp):
    # Split the timestamp into minutes, seconds, and milliseconds
    minutes, seconds, milliseconds = map(float, timestamp.split(':'))
    # Convert the time to seconds (including fractional part from milliseconds)
    total_seconds = minutes * 60 + seconds + milliseconds / 1000
    return total_seconds


def extract_segments(transcript):
    # Regular expression to match the timestamp and speaker tag
    timestamp_pattern = r'(\d{2}:\d{2}:\d{2})'
    speaker_pattern = r'\[([^\]]+)\]'

    lines = transcript.strip().splitlines()
    segments = []

    start_time = None
    speaker_tag = None

    for i in range(len(lines)):
        if re.match(timestamp_pattern, lines[i]):  # Line is a timestamp
            if start_time and speaker_tag:
                # If we have both start and speaker, the current timestamp is the end time
                end_time = convert_time_to_seconds(lines[i])
                segments.append((start_time, end_time, speaker_tag))
                start_time = None
                speaker_tag = None
            # Set the new start time, converting to seconds
            start_time = convert_time_to_seconds(lines[i])
        elif re.match(speaker_pattern, lines[i]):  # Line contains a speaker tag
            speaker_tag = re.findall(speaker_pattern, lines[i])[0]

    return segments

In [ ]:
#ensure new line before speaker tags
diarize_ds['transcript'] = diarize_ds['transcript'].apply(lambda x: str(x).replace('[', '\r\n['))
diarize_ds['ref_segments'] = diarize_ds['transcript'].apply(lambda x: extract_segments(x))

In [ ]:
diarize_ds.head()

,audio_id,path,transcript,age_group,gender,accent,domain,country,audio_duration,doctor_summary,all_timestamps,pred_segments,ref_segments
0,78fb7265-0373-49d5-b933-c28c95cf3eea,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:01:06\r\n\r\n[Speaker 1]: Are you there?\r\...,26-40,Male,Yoruba,OSCE-Doctor-Patient,NG,906.433991,"Mr. Ade, a 35 year old unmarried Yoruba male t...",True,"[(0.84, 3.96, Speaker 1), (3.96, 9.12, Speaker...","[(1.006, 1.027, Speaker 1), (4.0, 8.038, Speak..."
1,6d5f1e84-dbc5-42d3-898e-bfe9c2cfe166,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,"00:00:06\r\n\r\n[Speaker 1]: Good evening, I'm...",26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,204.622993,Johnny Dobra is a 35-year-old male who present...,True,"[(0.06, 2.76, Speaker 1), (2.76, 6.42, Speaker...","[(0.006, 2.028, Speaker 1), (2.049, 6.009, Spe..."
2,ab8fd39d-52bc-48fa-bae9-ff636627f1e5,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:00:08\r\n\r\n[Speaker 1]: My name is Nwachu...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,343.260000,"The patient, a 45 year old female accontant, p...",True,"[(2.88, 10.08, Speaker 1), (10.08, 11.94, Spea...","[(0.008, 2.04, Speaker 1), (2.049, 5.023, Spea..."
3,5129fd8c-7b8c-4d05-a03a-196bcae4deff,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:00:50\r\n\r\n[Speaker 1]: My name is Chiama...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,337.587982,"John Peter, a 45-year-old construction worker,...",True,"[(0.66, 4.14, Speaker 1), (4.14, 7.44, Speaker...","[(0.05, 3.044, Speaker 1), (4.016, 6.035, Spea..."
4,58efe9ef-6c76-4d17-b1b2-f397552a0c0e,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:01:13\r\n\r\n[Speaker 1]: Good day. I am Do...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,213.635986,"Peter, a 45-year-old male, presented to the ho...",True,"[(1.14, 5.58, Speaker 1), (5.58, 7.08, Speaker...","[(1.013, 5.006, Speaker 1), (5.028, 6.028, Spe..."


In [ ]:
from pyannote.core import Annotation, Segment
from pyannote.metrics.diarization import DiarizationErrorRate

def create_pyannote_annotation(segments_list):
    annotation = Annotation()
    for start, end, speaker_tag in segments_list:
        segment = Segment(start, end)
        annotation[segment] = speaker_tag
    return annotation

der_metric = DiarizationErrorRate()

In [ ]:
for i, text in tqdm(enumerate(diarize_ds['transcript']), total=len(diarize_ds['transcript']), desc="Processing"):
    ref_annotation = create_pyannote_annotation(diarize_ds.iloc[i]['ref_segments'])
    pred_annotation = create_pyannote_annotation(diarize_ds.iloc[i]['pred_segments'])
    der = der_metric(ref_annotation, pred_annotation)
    print(f"DER: {100 * der:.2f}%")
#get abs value for whole dataset
ds_der = abs(der_metric)
print(f"Absolute DER for dataset: {100 * ds_der:.2f}%")

Processing:   0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/pyannote/metrics/utils.py:200: UserWarning: 'uem' was approximated by the union of 'reference' and 'hypothesis' extents.
  warnings.warn(
Processing:   7%|▋         | 2/30 [00:00<00:04,  6.72it/s]

DER: 54.28%
DER: 36.99%


Processing:  10%|█         | 3/30 [00:00<00:03,  6.79it/s]

DER: 31.36%


Processing:  13%|█▎        | 4/30 [00:00<00:06,  4.00it/s]

...

DER: 39.08%
DER: 40.03%
DER: 3.40%
DER: 11.06%
DER: 48.95%
DER: 2.88%
DER: 4.62%


Processing:  87%|████████▋ | 26/30 [00:01<00:00, 32.50it/s]

DER: 8.69%
DER: 17.86%
DER: 3.11%
DER: 51.72%
DER: 43.33%
DER: 1.98%
DER: 5.09%
DER: 13.37%
DER: 3.92%
DER: 6.78%
DER: 25.52%
DER: 2.30%


Processing: 100%|██████████| 30/30 [00:01<00:00, 17.03it/s]

DER: 11.08%
DER: 21.55%
DER: 5.32%
DER: 50.35%
Absolute DER for dataset: 20.05%


In [ ]:
diarize_ds.to_csv('/content/drive/MyDrive/AfriSpeech-Dialog-data/diarization_results/soniox_diarization_der_0.2005_30.csv', index=False)